## FINAL PROJECT.

### Client: AtliQ Hardware.

### AtliQ Hardware is one of the leading computer hardware producers in India, and has even expanded into other countries as well. They sell different types of hardware to big players such as Amazon, Best Buy, and Walmart.<br>
### This year, they're asking PWC to conduct a big audit of their sales and help them automate their existing data.

***

### FINNANCIAL ANALYSIS

#### DECOMPOSITION AND RESEARCH TASKS:

### 1. Definition of Objectives and Key Questions

#### Objective: Understand how revenue, profits, and margins have changed over time. Analyze shifts in the market and identify the most profitable categories.<br>
#### Key Questions:<br>
- How have revenue and profits trended over the years?<br>
- What are the profit margins, and how have they evolved?<br>
- Which markets, platforms, or channels have driven the most revenue and profit?<br>
- What categories of products are the most profitable?<br>
- How have different regions/subzones contributed to revenue and profit?<br>
- Are there any noticeable shifts in market trends or customer preferences?

***

### 2. Identify the Important Metrics

#### Revenue Metrics:<br>
- Total Revenue: Sum of gross_price * sold_quantity from fact_sales_monthly.
- Revenue Growth Rate: Percentage increase/decrease in revenue year-over-year.
#### Profit Metrics:
- Gross Profit: Revenue minus the manufacturing cost (gross_price * sold_quantity - manufacturing_cost * sold_quantity).
- Profit Margin: Gross profit as a percentage of total revenue.
- Net Profit: Gross profit minus pre-invoice discounts (gross_profit - (pre_invoice_discount_pct * gross_profit)).
- Net Profit Margin: Net profit as a percentage of total revenue.
#### Market and Category Metrics:
- Revenue by Market: Revenue broken down by market.
- Revenue by Platform: Revenue broken down by platform.
- Profit by Category: Profitability of different segment and category.
#### Trend Metrics:<br>
- Market Share: Proportion of total revenue by market.
- Revenue/Profit by Year: Trend of revenue and profit over time (fiscal_year).

***

### 3. Data Preparation and Cleaning

#### Data Integrity Check:<br>
- Ensure each market belongs to one sub_zone and region.
- Validate that each customer has only one associated platform.
- Check that product_code is consistent across all fact tables.
- Verify that there are only 3 unique values in the division column of the dim_product table.
#### Handle Missing Data:<br>
- Identify and address missing values in key columns like gross_price, manufacturing_cost, and sold_quantity.
- Impute or remove records with missing data as appropriate.
#### Feature Engineering:<br>
- Create new features like gross_profit, net_profit, profit_margin, and net_profit_margin for further analysis.
- Aggregate data by fiscal_year, market, platform, etc., to facilitate trend analysis.

In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect('/Users/cesarchaparro/Desktop/TripleTen/Sprint_12/Final_Project/atliq_db.sqlite3')

In [4]:
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('dim_customer',), ('dim_product',), ('fact_pre_discount',), ('fact_manufacturing_cost',), ('fact_gross_price',), ('fact_sales_monthly',)]


#### OPEN EACH TABLE INDIVIDUALLY

#### 1. Customer table:

In [5]:
# opening the customer table
query = """Select * from 
dim_customer
"""
dim_customer = pd.read_sql_query(query, con)
dim_customer.head(10)

,customer_code,customer,platform,channel,market,sub_zone,region
0,70002017,Atliq Exclusive,Brick & Mortar,Direct,India,India,APAC
1,70002018,Atliq e Store,E-Commerce,Direct,India,India,APAC
2,70003181,Atliq Exclusive,Brick & Mortar,Direct,Indonesia,ROA,APAC
3,70003182,Atliq e Store,E-Commerce,Direct,Indonesia,ROA,APAC
4,70004069,Atliq Exclusive,Brick & Mortar,Direct,Japan,ROA,APAC
5,70004070,Atliq e Store,E-Commerce,Direct,Japan,ROA,APAC
6,70005163,Atliq e Store,E-Commerce,Direct,Pakistan,ROA,APAC
7,70006157,Atliq Exclusive,Brick & Mortar,Direct,Philiphines,ROA,APAC
8,70006158,Atliq e Store,E-Commerce,Direct,Philiphines,ROA,APAC
9,70007198,Atliq Exclusive,Brick & Mortar,Direct,South Korea,ROA,APAC


In [6]:
dim_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_code  209 non-null    int64 
 1   customer       209 non-null    object
 2   platform       209 non-null    object
 3   channel        209 non-null    object
 4   market         209 non-null    object
 5   sub_zone       209 non-null    object
 6   region         209 non-null    object
dtypes: int64(1), object(6)
memory usage: 11.6+ KB


In [7]:
query = """SELECT customer, COUNT(DISTINCT platform) AS platform_count
FROM dim_customer
GROUP BY customer
HAVING COUNT(DISTINCT platform) > 1;
"""
result = pd.read_sql_query(query, con)
print(result)

Empty DataFrame
Columns: [customer, platform_count]
Index: []


##### There are no customer who have more than one platform. Each customer has one platform.

In [8]:
query = """SELECT market, 
       COUNT(DISTINCT sub_zone) AS distinct_subzones, 
       COUNT(DISTINCT region) AS distinct_regions
FROM dim_customer
GROUP BY market
HAVING distinct_subzones > 1 OR distinct_regions > 1;
"""
result = pd.read_sql_query(query, con)
print(result)


Empty DataFrame
Columns: [market, distinct_subzones, distinct_regions]
Index: []


##### Each market has been checked as having only 1 subzone and 1 region under it.

***

#### 2. Product table:

In [9]:
# opening the product table
query = """Select * from 
dim_product
"""
dim_product = pd.read_sql_query(query, con)
dim_product.head()

,product_code,division,segment,category,product,variant
0,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard
1,A0118150102,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Plus
2,A0118150103,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Premium
3,A0118150104,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Premium Plus
4,A0219150201,P & A,Peripherals,Internal HDD,AQ WereWolf NAS Internal Hard Drive HDD – 8.89 cm,Standard


In [10]:
dim_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_code  397 non-null    object
 1   division      397 non-null    object
 2   segment       397 non-null    object
 3   category      397 non-null    object
 4   product       397 non-null    object
 5   variant       397 non-null    object
dtypes: object(6)
memory usage: 18.7+ KB


In [11]:
query = """SELECT COUNT(DISTINCT division) as unique_divisions 
FROM dim_product;
"""
result = pd.read_sql_query(query, con)
print(result)

if result['unique_divisions'][0] == 3:
    print("There are exactly 3 unique values in the 'division' column.")
else:
    print(f"There are {result['unique_divisions'][0]} unique values in the 'division' column.")

   unique_divisions
0                 3
There are exactly 3 unique values in the 'division' column.


##### After verifying the column I can conclude that there are only 3 types of divisions on the column.

In [12]:
query = """SELECT COUNT(DISTINCT segment) as unique_segments 
FROM dim_product;
"""
result = pd.read_sql_query(query, con)
print(result)

if result['unique_segments'][0] == 6:
    print("There are exactly 6 unique values in the 'segment' column.")
else:
    print(f"There are {result['unique_segments'][0]} unique values in the 'segment' column.")

   unique_segments
0                6
There are exactly 6 unique values in the 'segment' column.


##### After verifying the segments column I can conclude that there are only 6 types of segments on the column.

In [13]:
# checking the dim_product table for unique product codes
query = "SELECT DISTINCT product_code FROM dim_product"
dim_product_codes = pd.read_sql_query(query, con)
dim_product_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_code  397 non-null    object
dtypes: object(1)
memory usage: 3.2+ KB


#### After checking the unique codes for each product we can see that there are no duplicates, missing or abnormal codes. Each product code has a unique value.

#### Product codes are also used in other related tables, such as fact_manufacturing_cost, fact_gross_price, and fact_sales_monthly. I will check further on the other tables to see if there are duplicates, missing or abnormal codes.

***

#### 3. Discounts table:

In [14]:
# opening the discounts applied to customers table
query = """Select * from 
fact_pre_discount
"""
fact_pre_discount = pd.read_sql_query(query, con)
fact_pre_discount.head(10)

,customer_code,fiscal_year,pre_invoice_discount_pct
0,70002017,2018,0.0824
1,70002017,2019,0.0777
2,70002017,2020,0.0735
3,70002017,2021,0.0703
4,70002017,2022,0.1057
5,70002018,2018,0.2956
6,70002018,2019,0.2577
7,70002018,2020,0.2255
8,70002018,2021,0.2061
9,70002018,2022,0.2931


In [15]:
fact_pre_discount.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045 entries, 0 to 1044
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_code             1045 non-null   int64  
 1   fiscal_year               1045 non-null   int64  
 2   pre_invoice_discount_pct  1045 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 24.6 KB


In [16]:
# count the number of rows with a pre_invoice_discount_pct of 0
query = """SELECT COUNT(*)
FROM fact_pre_discount
WHERE pre_invoice_discount_pct = 0
"""
zero_discount = pd.read_sql_query(query, con)
zero_discount

,COUNT(*)
0,0


#### There are no products with discount value as 0.

#### 4. Costs table:

In [17]:
# opening the cost table
query = """Select * from 
fact_manufacturing_cost
"""
fact_manufacturing_cost = pd.read_sql_query(query, con)
fact_manufacturing_cost.head(10)

,product_code,cost_year,manufacturing_cost
0,A0118150101,2018,4.6190
1,A0118150101,2019,4.2033
2,A0118150101,2020,5.0207
3,A0118150101,2021,5.5172
4,A0118150102,2018,5.6036
5,A0118150102,2019,5.3235
6,A0118150102,2020,5.7180
7,A0118150102,2021,6.2835
8,A0118150103,2018,5.9469
9,A0118150103,2019,5.5306


In [18]:
fact_manufacturing_cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1182 entries, 0 to 1181
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_code        1182 non-null   object 
 1   cost_year           1182 non-null   int64  
 2   manufacturing_cost  1182 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 27.8+ KB


In [19]:
# count the number of rows with a manufacturing_cost of 0
query = """SELECT COUNT(*)
FROM fact_manufacturing_cost
WHERE manufacturing_cost = 0
"""
zero_manufacturing_cost = pd.read_sql_query(query, con)
zero_manufacturing_cost

,COUNT(*)
0,0


#### There are no products with cost value as 0.

#### There are 1182 values on the column. I will analyze further.

#### Product codes are also used in fact_manufacturing_cost table, so to check for consistency.

In [20]:
# count the frequency of each product_code
frequency_codes = pd.read_sql_query("""SELECT product_code, COUNT(*) AS frequency
FROM fact_manufacturing_cost
GROUP BY product_code;
""", con)
frequency_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389 entries, 0 to 388
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_code  389 non-null    object
 1   frequency     389 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.2+ KB


#### There are 389 unique value codes.

In [21]:
# to see how many product codes appear exactly once, more than once, etc.
more_frequent_codes = pd.read_sql_query("""SELECT frequency, COUNT(*) AS number_of_product_codes
FROM (
    SELECT product_code, COUNT(*) AS frequency
    FROM fact_manufacturing_cost
    GROUP BY product_code
) AS frequency_table
GROUP BY frequency
ORDER BY frequency DESC;
""", con)
more_frequent_codes

,frequency,number_of_product_codes
0,5,59
1,4,91
2,3,90
3,2,104
4,1,45


#### From this output we can see the distribution of product codes within the column, there are 59 products that appear 5 times, 91 products that appear 4 times, 90 products that appear 3 times, 104 products that appear 2 times and 45 that appear 1 time. Which gives a total of 1182 rows. Now there is an understanding of the column.

#### Now I will check if there are missing codes on this table that appear on the original table of products.

In [22]:
# check for missing product codes
missing_in_fact_manufacturing = dim_product_codes[~dim_product_codes['product_code'].isin(fact_manufacturing_cost['product_code'])]
print("Product codes in dim_product missing in fact_manufacturing_cost:")
print(missing_in_fact_manufacturing)

Product codes in dim_product missing in fact_manufacturing_cost:
    product_code
376  A6718160102
377  A6719160101
382  A6918160303
383  A6919160302
384  A6921160301
385  A7018160402
386  A7019160401
387  A7020160403


#### There are 8 codes that are missing in the fact_manufacturing_cost from the dim_product table.

#### Here are the whole rows of the missing codes to check for anomalies on the data.

In [23]:
missing_codes = ('A6718160102', 'A6719160101', 'A6918160303', 'A6919160302', 
                 'A6921160301', 'A7018160402', 'A7019160401', 'A7020160403')

query = f"""
SELECT * FROM dim_product
WHERE product_code IN {missing_codes}
"""

missing_product_info = pd.read_sql_query(query, con)
missing_product_info

,product_code,division,segment,category,product,variant
0,A6718160102,N & S,Storage,USB Flash Drives,AQ Pen Drive 2 IN 1,Plus
1,A6719160101,N & S,Storage,USB Flash Drives,AQ Pen Drive 2 IN 1,Standard
2,A6918160303,N & S,Storage,USB Flash Drives,AQ Ultra Dual 2.0,Premium
3,A6919160302,N & S,Storage,USB Flash Drives,AQ Ultra Dual 2.0,Plus
4,A6921160301,N & S,Storage,USB Flash Drives,AQ Ultra Dual 2.0,Standard
5,A7018160402,N & S,Storage,USB Flash Drives,AQ Ultra Dual 3.0,Plus
6,A7019160401,N & S,Storage,USB Flash Drives,AQ Ultra Dual 3.0,Standard
7,A7020160403,N & S,Storage,USB Flash Drives,AQ Ultra Dual 3.0,Premium


#### Apparently there are no anomalies on the data itself. I will analyze the situation further.

#### My conclusion will show on the last table below.

#### I want to ensure that there are no product codes in the fact_manufacturing_cost that do not exist in the dim_product table.

In [24]:
# check for product codes in fact_manufacturing_cost that are not in dim_product
extra_in_fact_manufacturing = fact_manufacturing_cost[~fact_manufacturing_cost['product_code'].isin(dim_product_codes['product_code'])]
print("Product codes in fact_manufacturing_cost not found in dim_product:")
print(extra_in_fact_manufacturing)

Product codes in fact_manufacturing_cost not found in dim_product:
Empty DataFrame
Columns: [product_code, cost_year, manufacturing_cost]
Index: []


#### There are in fact no product codes in the cost table that don't exist in the product table.

#### 5. Pricing table:

In [25]:
# opening the prices table
query = """Select * from 
fact_gross_price
"""
fact_gross_price = pd.read_sql_query(query, con)
fact_gross_price.head(10)

,product_code,fiscal_year,gross_price
0,A0118150101,2018,15.3952
1,A0118150101,2019,14.4392
2,A0118150101,2020,16.2323
3,A0118150101,2021,19.0573
4,A0118150102,2018,19.5875
5,A0118150102,2019,18.5595
6,A0118150102,2020,19.8577
7,A0118150102,2021,21.4565
8,A0118150103,2018,19.3630
9,A0118150103,2019,19.3442


In [26]:
fact_gross_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1182 entries, 0 to 1181
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_code  1182 non-null   object 
 1   fiscal_year   1182 non-null   int64  
 2   gross_price   1182 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 27.8+ KB


In [27]:
# count the number of rows with a gross_price of 0
query = """SELECT COUNT(*)
FROM fact_gross_price
WHERE gross_price = 0
"""
zero_gross_price = pd.read_sql_query(query, con)
zero_gross_price

,COUNT(*)
0,0


#### There are no rows with value 0 as price. Therefore I conclude that the price values are correct.

#### Product codes are also used in fact_gross_price table.

In [28]:
# query to fetch all unique product codes from fact_gross_price
query = "SELECT DISTINCT product_code FROM fact_gross_price"
fact_gross_price_codes = pd.read_sql_query(query, con)
fact_gross_price_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389 entries, 0 to 388
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_code  389 non-null    object
dtypes: object(1)
memory usage: 3.2+ KB


In [29]:
# check for missing product codes
missing_in_fact_gross_price = dim_product_codes[~dim_product_codes['product_code'].isin(fact_gross_price_codes['product_code'])]
print("Product codes in dim_product missing in fact_gross_price:")
print(missing_in_fact_gross_price)

Product codes in dim_product missing in fact_gross_price:
    product_code
376  A6718160102
377  A6719160101
382  A6918160303
383  A6919160302
384  A6921160301
385  A7018160402
386  A7019160401
387  A7020160403


#### The same codes missing on the cost table are also missing on the price table.

#### I want to ensure that there are no product codes in the fact_gross_price that do not exist in the dim_product table.

In [30]:
# check for product codes in fact_gross_price that are not in dim_product
extra_in_fact_gross_price = fact_gross_price_codes[~fact_gross_price_codes['product_code'].isin(dim_product_codes['product_code'])]
print("Product codes in fact_gross_price not found in dim_product:")
print(extra_in_fact_gross_price)

Product codes in fact_gross_price not found in dim_product:
Empty DataFrame
Columns: [product_code]
Index: []


#### There are in fact no product codes in the price table that don't exist in the product table.

#### 6. Sales table:

In [31]:
# opening the sales table
query = """Select * from 
fact_sales_monthly
"""
fact_sales_monthly = pd.read_sql_query(query, con)
fact_sales_monthly.head(10)

,date,product_code,customer_code,sold_quantity,fiscal_year
0,2017-09-01,A0118150101,70002017,51,2018
1,2017-09-01,A0118150101,70002018,77,2018
2,2017-09-01,A0118150101,70003181,17,2018
3,2017-09-01,A0118150101,70003182,6,2018
4,2017-09-01,A0118150101,70006157,5,2018
5,2017-09-01,A0118150101,70006158,7,2018
6,2017-09-01,A0118150101,70007198,29,2018
7,2017-09-01,A0118150101,70007199,34,2018
8,2017-09-01,A0118150101,70008169,22,2018
9,2017-09-01,A0118150101,70008170,5,2018


In [32]:
fact_sales_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67250 entries, 0 to 67249
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           67250 non-null  object
 1   product_code   67250 non-null  object
 2   customer_code  67250 non-null  int64 
 3   sold_quantity  67250 non-null  int64 
 4   fiscal_year    67250 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 2.6+ MB


#### Product codes are also used in fact_sales_monthly table.

In [33]:
# query to check for product codes with multiple counts
duplicate_product_codes = pd.read_sql_query("""
SELECT product_code, COUNT(*) AS frequency
FROM fact_sales_monthly
GROUP BY product_code
HAVING COUNT(*) > 1
ORDER BY frequency DESC;
""", con)
print(duplicate_product_codes)

   product_code  frequency
0   A0118150104       6643
1   A0118150103       6635
2   A0418150103       6593
3   A0118150101       6005
4   A0118150102       6003
5   A0219150202       5747
6   A0219150201       5745
7   A0220150203       4318
8   A0320150301       4314
9   A0418150101       4100
10  A0418150102       4097
11  A0321150303       2508
12  A0321150302       2508
13  A0418150104       2034


In [34]:
# query to sum frequencies of product codes that appear more than once
total_frequency = pd.read_sql_query("""
SELECT SUM(frequency) AS total_frequency
FROM (
    SELECT product_code, COUNT(*) AS frequency
    FROM fact_sales_monthly
    GROUP BY product_code
    HAVING COUNT(*) > 1
) AS frequency_table;
""", con)
# Print the total frequency
print(total_frequency)

   total_frequency
0            67250


#### If we sum up each frequency we have the total of rows of 67250.

#### Now I will check if there are missing codes on this table that appear on the original table of products.

In [38]:
query = f"""
SELECT *
FROM dim_product
WHERE product_code IN ({','.join('?' for _ in missing_codes)});
"""
missing_codes_df = pd.read_sql_query(query, con, params = missing_codes)
missing_codes_df

,product_code,division,segment,category,product,variant
0,A6718160102,N & S,Storage,USB Flash Drives,AQ Pen Drive 2 IN 1,Plus
1,A6719160101,N & S,Storage,USB Flash Drives,AQ Pen Drive 2 IN 1,Standard
2,A6918160303,N & S,Storage,USB Flash Drives,AQ Ultra Dual 2.0,Premium
3,A6919160302,N & S,Storage,USB Flash Drives,AQ Ultra Dual 2.0,Plus
4,A6921160301,N & S,Storage,USB Flash Drives,AQ Ultra Dual 2.0,Standard
5,A7018160402,N & S,Storage,USB Flash Drives,AQ Ultra Dual 3.0,Plus
6,A7019160401,N & S,Storage,USB Flash Drives,AQ Ultra Dual 3.0,Standard
7,A7020160403,N & S,Storage,USB Flash Drives,AQ Ultra Dual 3.0,Premium


#### Conclusion about the 8 missing products on several tables:<br>
#### - These products could have been part of the product catalog but were either discontinued or never actually sold or stocked. This might be a result of these products being introduced but then removed from the system before any transactions could occur.<br>
#### - There may be a data entry or integration issue where these products were incorrectly entered into the dim_product table but were never correctly linked to any transactions or cost records.<br>
#### - It’s possible that these products were planned for introduction but were never actually put into production or sold. This could be due to strategic business decisions, market demand assessments, or other operational reasons.<br>
#### - There might be flaws in the data entry or processing systems that resulted in these products not being properly linked across tables.

#### I will not delete these produc_codes from the dataframe since it doesn't affect or influence my analysis on finnances and I don't see any anomalies on the data itself.

#### I will check the presence of more missing product codes in this table that appear on the original product table.

In [39]:
# define the list of product codes to exclude
excluded_product_codes = [
    'A6718160102', 
    'A6719160101', 
    'A6918160303', 
    'A6919160302', 
    'A6921160301', 
    'A7018160402', 
    'A7019160401', 
    'A7020160403'
]
# create a DataFrame with product codes to exclude
excluded_codes_df = pd.DataFrame({
    'product_code': excluded_product_codes
})
filtered_missing_products = fact_sales_monthly[~fact_sales_monthly['product_code'].isin(excluded_codes_df['product_code'])]
filtered_missing_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67250 entries, 0 to 67249
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           67250 non-null  object
 1   product_code   67250 non-null  object
 2   customer_code  67250 non-null  int64 
 3   sold_quantity  67250 non-null  int64 
 4   fiscal_year    67250 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 2.6+ MB


#### I asume that products that are newly added to the inventory may not yet have any sales recorded. If these products have been recently introduced, there might be no sales data for them yet.

***

### 4. Exploratory Data Analysis (EDA)

#### Descriptive Statistics:<br>
- Calculate mean, median, and standard deviation for revenue, profit, and margin metrics.
- Identify outliers or unusual trends in the data.
#### Trend Analysis:<br>
- Plot revenue, profit, and margin over time to visualize trends.
- Use time series analysis to forecast future revenue and profit.
#### Segment Analysis:<br>
- Break down revenue and profit by platform, market, region, and category.
- Identify which segments contribute most to overall profitability.
#### Correlation Analysis:<br>
- Analyze the relationship between different metrics (e.g., how does discount percentage affect net profit?).
- Look for correlations between market conditions and profitability.

***

### 5. Identifying Problems or Questions to Address

#### Profitability Issues:<br>
- Are certain markets or platforms less profitable? If so, why?
- Is there a declining trend in any key categories or segments?
#### Market Shifts:<br>
- Are there shifts in revenue from one platform to another (e.g., from Brick & Mortar to E-Commerce)?
- Are there emerging markets or declining markets based on the data?
#### Cost Efficiency:<br>
- Are manufacturing costs rising, and how does this affect profit margins?
- Are discounts eroding profitability in certain markets or for certain customers?

***

### 6. Research and Contextual Understanding

#### Industry Benchmarks:<br>
- Research industry standards for profit margins, cost structures, and market share to compare with your findings.
#### Economic Factors:<br>
- Consider external economic factors that may have influenced market trends (e.g., currency fluctuations, economic downturns).
#### Competitor Analysis:
- Research competitors' performance in similar markets to understand potential threats or opportunities.


***

### 7. Hypothesis Formation and Testing

#### Based on the EDA, form hypotheses about what drives revenue and profit. For example:<br>
- "E-Commerce platform is more profitable due to lower overhead costs."
- "The market in Japan has seen a decline due to increased competition."
#### Design experiments or statistical tests to validate these hypotheses.

***

### 8. Visualization and Reporting

#### Dashboards:<br>
- Create visualizations to represent the trends, comparisons, and insights derived from the data.
- Use tools like Power BI, Tableau, or Python libraries like Matplotlib/Seaborn.
#### Report:<br>
- Compile a report summarizing your findings, insights, and recommendations based on the analysis.
- Ensure that the report answers the key questions posed at the beginning of the project.

***

### 9. Actionable Recommendations

#### Based on the analysis, provide recommendations for business strategies to improve revenue, optimize profits, and adapt to market shifts. These could include:<br>
- Focusing more on profitable platforms or markets.
- Reducing costs in certain product lines.
- Increasing marketing efforts in emerging markets or declining segments.